In [5]:
#!pip install fastai

In [7]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
import re

# fixes a dependancy on PosixPath by load_learner
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

import random as rnd
from fastai import *
from fastai.vision.all import *

app = Flask(__name__)
if not os.path.isdir('UploadFolder'): os.mkdir('UploadFolder')
app.config['UPLOAD_FOLDER'] = './static/UploadFolder'

# These are the categories used in the model and the data
DATAPATH = "./static/data/"
CATEGORIES = os.listdir(DATAPATH)
ALLOWED_IMGTYPES = ['png','jpeg','gif','jfif','jpg']
# Every category has at least 600 images
N_MAX = 20

sample = {"sample_class":"sample_hidden"}
 
learn = load_learner('exportBig.pkl')

def format_example(imgpath,cat,prediction):
   return {"image":"/data/"+imgpath,
                           "category":cat,
                           "prediction":prediction[0],
                           "correct_class":"example_"+str(cat==prediction[0])}

def format_sample(imgpath,prediction):
   return {"image":"/UploadFolder/"+imgpath,
            "prediction":prediction[0],
            "sample_class":"sample_visable"}

def get_examples():
   examples = []
   for cat in CATEGORIES:
      n = rnd.choice(range(N_MAX))
      imgpath = cat+"/"+cat+str(n)+".png"
      prediction = learn.predict(DATAPATH+imgpath)

      examples.append(format_example(imgpath,cat,prediction))
   
   return examples


   
    


@app.route('/')
def home():
   global sample
   print(sample)
   examples = get_examples()
   return render_template('home.html',examples=examples,sample=sample)

@app.route("/testImage_post", methods=['post'])
def testImage_post():




   # for img in os.listdir(app.config['UPLOAD_FOLDER']):
   #    os.remove(os.path.join(app.config['UPLOAD_FOLDER'],img))
   global sample
   # examples are manualy set for now, straight from the model? or select some to test and test them and than compare to name using regex?

   image = request.files['file']
   #regex function to see if uploaded file was an image

   imgpath = secure_filename(image.filename)

   print("Image path:" + imgpath)
   
   

   if re.sub('.*\\.','',imgpath) not in ALLOWED_IMGTYPES:
      return redirect(url_for('home'))

   image.save(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   #test the posted image with the model
   prediction = learn.predict(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   sample = format_sample(imgpath,prediction)



   return redirect(url_for('home'))



if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'sample_class': 'sample_hidden'}


127.0.0.1 - - [12/Dec/2021 13:38:26] "GET / HTTP/1.1" 200 -


{'sample_class': 'sample_hidden'}


...

127.0.0.1 - - [12/Dec/2021 13:56:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 13:56:10] "GET /static/data/ancient%20egyptian/ancient%20egyptian10.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:10] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/federal/federal6.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/pictures/Stars.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/moorisch/moorisch11.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/roman/roman0.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/modern/modern9.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/gothic/gothic7.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/tudor/tudor8.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:56:11] "GET /static/data/art%20deco/art%20deco2.png HTTP/1.1" 200 -
127.0.0.1 - - [12/D

Image path:
{'image': '/UploadFolder/OnzeLieveVrouwetoren_Amersfoort-683x1024.jpg', 'prediction': 'gothic', 'sample_class': 'sample_visable'}


127.0.0.1 - - [12/Dec/2021 13:58:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/pictures/Stars.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/tudor/tudor8.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/gothic/gothic11.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/modern/modern9.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/moorisch/moorisch13.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/roman/roman14.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/federal/federal7.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static//data/cunstructivist/cunstructivist5.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 13:58:23] "GET /static/data/ancient%20egyptian/ancient%20egyptian2.png HTTP/1.1" 304 -
127.0.0.1

{'image': '/UploadFolder/OnzeLieveVrouwetoren_Amersfoort-683x1024.jpg', 'prediction': 'gothic', 'sample_class': 'sample_visable'}


127.0.0.1 - - [12/Dec/2021 14:01:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 14:01:44] "GET /static/data/ancient%20egyptian/ancient%20egyptian1.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:44] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/federal/federal8.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/pictures/Stars.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/indoislamic/indoislamic14.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/gothic/gothic3.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/tudor/tudor19.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/roman/roman19.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/art%20deco/art%20deco8.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 14:01:45] "GET /static/data/baroque/baroque16.png HTTP/1.1" 304 -
127.0.0.1